In [33]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
raw_data_dir = Path.cwd() / "data" / "raw"
clean_data_dir = Path.cwd() / "data" / "clean"

# Taylor Rule

In [4]:
time_span = [year for year in range(1980, 2003)]

### Interest Rates

(OECD)

In [5]:
df = pd.read_csv(
    raw_data_dir / "ST_interestrate_1980-2002.csv",
    sep=";",  
)

In [6]:
df = df.rename(columns={"Year": "year", "Interest Rate": "interest_rate"})

In [7]:
df = df.loc[df["year"].isin(time_span)]

In [8]:
df["interest_rate"] = df["interest_rate"].apply(lambda x: x.replace(",", ".")).astype(float)

In [9]:
df.to_csv(clean_data_dir / "interest_rate.csv", sep="\t", index=False)

### Inflation

(WWW.RIVALUTA.it)

In [121]:
df = pd.read_csv(
    raw_data_dir / "datiinflazionemediaitalia.csv", 
    sep=";",  
)

In [122]:
df = df[1:-1]

In [123]:
df = df.rename(columns={"WWW.RIVALUTA.it": "inflation_rate"})

In [124]:
df.reset_index(col_level=0, names="year", inplace=True)

In [125]:
df = df.sort_values("year")

In [126]:
df["year"] = df["year"].astype(int)

In [127]:
df = df.loc[df["year"].isin(time_span)]

In [128]:
df["inflation_rate"] = df["inflation_rate"].apply(
    lambda x: float(x.replace(",", ".").strip("%")) / 100.0
)

In [129]:
df

,year,inflation_rate
43,1980,0.212
42,1981,0.178
41,1982,0.165
40,1983,0.147
39,1984,0.108
38,1985,0.092
37,1986,0.058
36,1987,0.048
35,1988,0.050
34,1989,0.063


In [19]:
df.to_csv(clean_data_dir / "inflation_rate.csv", sep="\t", index=False)

### Inflation Target

(approximation reading papers)

The idea of targeting inflation has developed in the early 2000's once the ECB was introduced. 
Therefore, no clear-cut data was found. However, we know that the Italy was aiming at an inflation of about 5-7 % in the early 80's which started decerasing during the course of the years down to the iconic 2 %. 

Down here is the data estimated by Mario Sarcinelli.

In [133]:
years = list(range(1980, 2003))

inflation_target = [12, 11.5, 10.5, 10, 10, 7, 6, 4, 4.5, 4, 4.5, 5, 4.5, 4.5, 3.5, 4.7, 4., 3, 2, 2, 2, 2, 2]

df = pd.DataFrame({'year': years, 'inflation_target': inflation_target})

df.to_csv(clean_data_dir / "inflation_target.csv", sep="\t", index=False)

### Gdp

(Macrotrends)

In [20]:
df = pd.read_csv(
    raw_data_dir / "italy-gdp-gross-domestic-product.csv",
    sep=",",
    usecols=["date", "GDP ( Billions of US $)"],
)

In [21]:
df = df.rename(columns={"GDP ( Billions of US $)": "gdp"})

In [22]:
def date_to_year(date):
    date_object = datetime.strptime(date, "%Y-%m-%d")
    year = date_object.year
    return year

In [23]:
df["year"] = df["date"].apply(lambda x: date_to_year(x))
df.drop("date", axis=1, inplace=True)

In [24]:
df = df[["year", "gdp"]]

In [25]:
df = df.loc[df["year"].isin(time_span)]

In [26]:
df.head()

,year,gdp
20,1980,477.256776
21,1981,430.702851
22,1982,427.272646
23,1983,443.042374
24,1984,437.887689


In [27]:
df.to_csv(clean_data_dir / "gdp.csv", sep="\t", index=False)

### Output Gap 

(OECD, Nasdaq)

In [87]:
# work on oecd
df_oecd = pd.read_csv(
    raw_data_dir / "EO108_INTERNET_27032023112456648.csv",
    sep=",",
    usecols=["Value", "Time"]
)

# fill gaps with nasdaq
df_nasdaq = pd.read_csv(
    raw_data_dir / "nasdaq_output_gap.csv",
    sep=",",
)

In [88]:
df_oecd = df_oecd.rename(columns={"Time": "year", "Value": "output_gap"})

In [89]:
df_oecd.head()

,year,output_gap
0,1985,-2.239437
1,1986,-1.493276
2,1987,-0.898263
3,1988,0.883434
4,1989,1.800495


In [90]:
df_oecd["year"].unique()
# need from 1980 to 1984

array([1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021, 2022])

In [91]:
df_oecd = df.loc[df_oecd["year"].isin(time_span)]

In [92]:
df_nasdaq = df_nasdaq.rename(columns={"Value": "output_gap"})

In [93]:
df_nasdaq["year"] = df_nasdaq["Date"].apply(lambda x: date_to_year(x))
df_nasdaq.drop("Date", axis=1, inplace=True)

In [94]:
df_nasdaq = df_nasdaq[["year", "output_gap"]]

In [95]:
df_nasdaq = df_nasdaq.sort_values("year")

In [96]:
df_nasdaq = df_nasdaq.loc[df_nasdaq["year"].isin([year for year in range(1980, 1985)])]

In [100]:
df = pd.concat([df_nasdaq, df_oecd])

In [101]:
df.to_csv(clean_data_dir / "output_gap.csv", sep="\t", index=False)

# Additional Regressors

### Unemployment

(FRED)

In [67]:
df = pd.read_csv(
    raw_data_dir / "LRHUTTTTITA156S.csv",
    sep=",",
)

In [68]:
df = df.rename(columns={"DATE": "year", "LRHUTTTTITA156S": "unemployment_rate"})

In [70]:
df["year"] = df["year"].apply(lambda x: date_to_year(x))

In [73]:
df = df.loc[df["year"].isin(time_span)]

In [74]:
df.head()

,year,unemployment_rate
0,1982,7.166667
1,1983,7.366667
2,1984,7.833333
3,1985,8.141667
4,1986,8.883333


In [75]:
df.to_csv(clean_data_dir / "unemployment_rate.csv", sep="\t", index=False)

### Exchange Rate

In [56]:
df = pd.read_csv(
    raw_data_dir / "USD_serie_storica_giornaliera .csv",
    sep=",",
    usecols=["Data di Riferimento", "Quotazione"]
)

In [57]:
df = df.rename(columns={"Data di Riferimento": "year", "Quotazione": "exchange_rate"})

In [58]:
df = df.sort_values("year")

In [59]:
df["year"] = df["year"].apply(lambda x: date_to_year(x))

In [60]:
df = df.drop_duplicates("year", keep='last')

In [61]:
df = df.loc[df["year"].isin(time_span)]

In [62]:
add = pd.DataFrame(
[[2002, 1950.000]],
    columns=["year", "exchange_rate"]
)
add

,year,exchange_rate
0,2002,1950.0


In [64]:
df = pd.concat([df, add])

In [66]:
df = df.reset_index(drop=True)

In [67]:
df.to_csv(clean_data_dir / "exchange_rate.csv", sep="\t", index=False)

### Foreign Interest Rate

In [31]:
df = pd.read_csv(
    raw_data_dir / "US10y_bond.csv",
    sep=",",
)

In [32]:
df = df.rename(columns={"YEAR": "year", "Value": "US_interest_rate"})

In [33]:
df = df.loc[df["year"].isin(time_span)]

In [34]:
df.head()

,year,US_interest_rate
20,1980,11.460000
21,1981,13.910833
22,1982,13.001667
23,1983,11.105000
24,1984,12.438333


In [35]:
df.to_csv(clean_data_dir / "us_bond_yield.csv", sep="\t", index=False)